# DBMaint
Will show how to perform some remote dbmaint operations. This notebook will use the two clsuters created in the [create_all](create_all.ipynb) notebook to add a column to a table in the database, then use the second cluster to show the table's state (before and after).

## Outline
1. Have the dbmaint_cluster_maint cluster load the database   
2. Add a Price column to the table example using the dbmaint function addcol   
3. Commit the changes using the cluster function .aws.commit_kx_database   
4. Update the dataview to present the latest version of the database   
5. Show the before and after state of the table on the query cluster   
    a. Schema before updating the dataview   
    b. Update the cluster's database (use updated view)   
    c. Schema after using the updated view   

## Architecture
<img src="images/Deepdive Diagrams-dbmaint.drawio.png"  width="50%">

## References
[FinSpace DBMaint Documentation](https://docs.aws.amazon.com/finspace/latest/userguide/finspace-managed-kdb-databases-dbmaint.html)   
[DBMaint (github)](https://github.com/KxSystems/kdb/blob/master/utils/dbmaint.md)


In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import boto3
import json
import datetime

import pykx as kx

from env import *
from config import *
from managed_kx import *

# set q console width and height
kx.q.system.display_size = [50, 1000]

# ----------------------------------------------------------------

In [2]:
# create finspace client
session = boto3.Session()
client = get_client(session=session)

# Clusters Used in this Example
These are the cluseter we will use. the maint cluster makes the changes, the query cluster is used to show the before/after table states once dbamint is performed.

In [3]:
cdf = get_clusters(client, environmentId=ENV_ID)

if cdf is not None:
    cdf = cdf[cdf['clusterName'].isin(all_clusters)]

display(cdf)

,clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
6,dbmaint_cluster_maint,RUNNING,GP,None,"[{'key': 's', 'value': '4'}, {'key': 'AWS_ZIP_DEFAULT', 'value': '17,2,6'}]",dbmaint_cluster_maint cluster created with create_all notebook,2024-11-12 17:32:28.564000+00:00,2024-11-12 17:19:31.732000+00:00,dbmaintdb,None
7,dbmaint_cluster_query,RUNNING,GP,None,"[{'key': 's', 'value': '4'}, {'key': 'AWS_ZIP_DEFAULT', 'value': '17,2,6'}]",dbmaint_cluster_query cluster created with create_all notebook,2024-11-12 17:31:44.672000+00:00,2024-11-12 17:19:35.298000+00:00,dbmaintdb,None


# Add a Price Column
Using a dbmaint function (addcol), add a column to the example table of the database.

In [4]:
# connection to dbmaint cluster
conn = get_pykx_connection(client, 
    environmentId=ENV_ID, clusterName=MAINT_CLUSTER_NAME, 
    userName=KDB_USERNAME, boto_session=session)

In [5]:
# get the connection string to the cluster
conn_str = get_kx_connection_string(client, 
                          environmentId=ENV_ID, clusterName=MAINT_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

# parse the connection string to components
host, port, username, password = parse_connection_string(conn_str)


## Current Schema
This is the schema of the example table before the column is added.

In [6]:
%%q --host $host --port $port --user $username --pass $password

/ load the database
.Q.l `$.aws.akdbp, "/", .aws.akdb

/ before schema
meta example

c     | t f a
------| -----
date  | d    
sym   | s   p
time  | p    
number| j    


## Add a Price Column
Using the addcol function from dbmaint, add a price column to the table, set the default value of price to 0. 

.aws.akdbp: database path    
.aws.akdb: database name.  

In [7]:
%%q --host $host --port $port --user $username --pass $password
\cd /opt/kx/app/db

/ add the column
addcol[`:dbmaintdb;`example;`price;0h]

meta example

c     | t f a
------| -----
date  | d    
sym   | s   p
time  | p    
number| j    
price | h    


## Commit Changes
Using the .aws.commit_kx_database function (available from kdb clusters), commit changes made to the table to the managed database. This function will sweep the local database for all changes and add them to the managed database as one changeset. 

In [8]:
# commit changes, use pykx so we can capture the new chanegtset ID and poll for its status
res = conn(f'.aws.commit_kx_database["{DB_NAME}"]')

# was there an error or not?
CHANGESET_ID = res.get("id", None)

if CHANGESET_ID is None:
    display(res)
else:
    # get the changeset, wait for it to finish loading
    CHANGESET_ID = str(CHANGESET_ID)
    # wait for ingestion of changeset
    wait_for_changeset_status(client, environmentId=ENV_ID, databaseName=DB_NAME, changesetId=CHANGESET_ID, show_wait=True)
    print("** Done **")

Status is IN_PROGRESS, total wait 0:00:00, waiting 10 sec ...
Status is IN_PROGRESS, total wait 0:00:10, waiting 10 sec ...
** Done **


## DB Schema on Query Cluster
Al this while, the query cluster has been serviing up the database before changes (static view, initial changeset/version). 
This is the schema still on the query cluster, this is before updating to the new changes.

In [9]:
# Connect to the query cluster
conn = get_pykx_connection(client, 
    environmentId=ENV_ID, clusterName=QUERY_CLUSTER_NAME, 
    userName=KDB_USERNAME, boto_session=session)

# schema of table in query cluster
display( conn("meta example").pd() )

,t,f,a
c,,,
date,b'd',,
sym,b's',,p
time,b'p',,
number,b'j',,


## Update Query Cluster's Dataview
Now update the query cluster's dataview to the latest changeset_id added from the dbmaint cluster. This new version of the database will include the change made to the example table. 

When updating the clsuter to the new version (changeset_id) there are two steps to execute: 
1. Update the dataview to use the new version
2. Update the database/dataview used by the query cluster.

In [10]:
# update the query cluster's dataview to include the newly added changeset.
resp = client.update_kx_dataview(environmentId=ENV_ID, 
    databaseName=DB_NAME, 
    dataviewName=QUERY_DBVIEW_NAME, 
    changesetId=CHANGESET_ID, 
    segmentConfigurations=[
        {'dbPaths': ['/*'], 'volumeName': VOLUME_NAME}
    ]
)

In [11]:
# Dataview will go from UPDATING to ACTIVE once updates are complete
wait_for_dataview_status(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=QUERY_DBVIEW_NAME, show_wait=True)

Dataview: dbmaintdb_DBVIEW_QUERY status is UPDATING, total wait 0:00:00, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_QUERY status is UPDATING, total wait 0:00:30, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_QUERY status is UPDATING, total wait 0:01:00, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_QUERY status is UPDATING, total wait 0:01:30, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_QUERY status is UPDATING, total wait 0:02:00, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_QUERY status is UPDATING, total wait 0:02:30, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_QUERY status is UPDATING, total wait 0:03:00, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_QUERY status is UPDATING, total wait 0:03:30, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_QUERY status is UPDATING, total wait 0:04:00, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_QUERY status is UPDATING, total wait 0:04:30, waiting 30 sec ...
Dataview: dbmaintdb_DBVIEW_QUERY status is UPDATING, total wait 0:05:00, waiting

{'databaseName': 'dbmaintdb',
 'dataviewName': 'dbmaintdb_DBVIEW_QUERY',
 'azMode': 'SINGLE',
 'availabilityZoneId': 'use1-az6',
 'changesetId': 'zsmQulnawKXiOgwBUFFmjA',
 'segmentConfigurations': [{'dbPaths': ['/*'],
   'volumeName': 'DBMAINT_VOLUME',
   'onDemand': False}],
 'activeVersions': [{'changesetId': 'zsmQulnawKXiOgwBUFFmjA',
   'segmentConfigurations': [{'dbPaths': ['/*'],
     'volumeName': 'DBMAINT_VOLUME',
     'onDemand': False}],
   'attachedClusters': [],
   'createdTimestamp': datetime.datetime(2024, 11, 12, 17, 38, 59, 87000, tzinfo=tzlocal()),
   'versionId': 'PMmQuoXH8E7mIVimA5QGOw'},
  {'changesetId': 'VsmQr8Rc9WND6QaFsw2bdg',
   'segmentConfigurations': [{'dbPaths': ['/*'],
     'volumeName': 'DBMAINT_VOLUME',
     'onDemand': False}],
   'attachedClusters': ['dbmaint_cluster_query'],
   'createdTimestamp': datetime.datetime(2024, 11, 12, 17, 15, 54, 486000, tzinfo=tzlocal()),
   'versionId': 'QMmQrXV7eDSjXTBkfClBAA'}],
 'description': 'Dataview of database dbma

In [12]:
# Update the database/dataview on the cluster to use the new one
resp=client.update_kx_cluster_databases(environmentId=ENV_ID, 
    clusterName=QUERY_CLUSTER_NAME, 
    databases=[
        {'databaseName': DB_NAME, 'dataviewName': QUERY_DBVIEW_NAME}
    ],
    deploymentConfiguration={
        'deploymentStrategy': 'NO_RESTART'
    }
)

In [13]:
# Cluster will go from UPDATING to RUNNING once updates are complete
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=QUERY_CLUSTER_NAME, show_wait=True)

Cluster: dbmaint_cluster_query status is UPDATING, total wait 0:00:00, waiting 30 sec ...
Cluster: dbmaint_cluster_query status is UPDATING, total wait 0:00:30, waiting 30 sec ...
Cluster: dbmaint_cluster_query status is now RUNNING, total wait 0:01:00


{'status': 'RUNNING',
 'clusterName': 'dbmaint_cluster_query',
 'clusterType': 'GP',
 'volumes': [{'volumeName': 'DBMAINT_VOLUME', 'volumeType': 'NAS_1'}],
 'databases': [{'databaseName': 'dbmaintdb',
   'dataviewConfiguration': {'dataviewName': 'dbmaintdb_DBVIEW_QUERY',
    'dataviewVersionId': 'PMmQuoXH8E7mIVimA5QGOw',
    'changesetId': 'zsmQulnawKXiOgwBUFFmjA',
    'segmentConfigurations': [{'dbPaths': ['/*'],
      'volumeName': 'DBMAINT_VOLUME',
      'onDemand': False}]}}],
 'clusterDescription': 'dbmaint_cluster_query cluster created with create_all notebook',
 'releaseLabel': '1.0',
 'vpcConfiguration': {'vpcId': 'vpc-0fe2b9c50f3ad382f',
  'securityGroupIds': ['sg-0c99f1cfb9c3c7fd9'],
  'subnetIds': ['subnet-04052219ec25b062b'],
  'ipAddressType': 'IP_V4'},
 'initializationScript': 'initdb.q',
 'commandLineArguments': [{'key': 's', 'value': '4'},
  {'key': 'AWS_ZIP_DEFAULT', 'value': '17,2,6'}],
 'code': {'s3Bucket': 'kdb-demo-829845998889-kms',
  's3Key': 'code/dbmaint.zip'},

## Query Cluster with Latest Schema
Now demonstrate the query cluster has the updated schema of the example table.

In [14]:
# Re-Connect to the query cluster
conn = get_pykx_connection(client, 
    environmentId=ENV_ID, clusterName=QUERY_CLUSTER_NAME, 
    userName=KDB_USERNAME, boto_session=session)

# re-load the database
conn('.Q.l `$.aws.akdbp,"/",.aws.akdb')

# new schema in query
display( conn("meta example").pd() )

# sample
display( conn("select from example").pd() )


,t,f,a
c,,,
date,b'd',,
sym,b's',,p
time,b'p',,
number,b'j',,
price,b'h',,


,date,sym,time,number,price
0,2023-04-14,aaa,2023-04-14 23:17:28.473,53231,0
1,2023-04-14,aaa,2023-04-14 23:17:28.473,153560,0
2,2023-04-14,aaa,2023-04-14 23:17:28.473,449428,0
3,2023-04-14,aaa,2023-04-14 23:17:28.473,631966,0
4,2023-04-14,aaa,2023-04-14 23:17:28.473,941566,0
...,...,...,...,...,...
9999995,2023-04-23,ppp,2023-04-23 23:17:21.799,946617,0
9999996,2023-04-23,ppp,2023-04-23 23:17:21.799,249468,0
9999997,2023-04-23,ppp,2023-04-23 23:17:21.799,634620,0
9999998,2023-04-23,ppp,2023-04-23 23:17:21.799,855402,0


In [15]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-11-12 17:46:40.317920
